In [410]:
import sqlite3
import pandas as pd

In [411]:
con = sqlite3.connect(':memory:')

In [412]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [413]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [414]:
def get_data(query):
    df=pd.read_sql(
    f'''
    {query}
    ''',
    con)
    return(df)

In [417]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [418]:
tasks+=1
get_data(
    '''
 SELECT * FROM Customers
 WHERE Country IN ('Germany', 'France') OR City = 'Madrid'
    ''')

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,6,Blauer See Delikatessen,Hanna Moos,Forsterstr. 57,Mannheim,68306,Germany
2,7,Blondel père et fils,Frédérique Citeaux,"24, place Kléber",Strasbourg,67000,France
3,8,Bólido Comidas preparadas,Martín Sommer,"C/ Araquil, 67",Madrid,28023,Spain
4,9,Bon app',Laurence Lebihans,"12, rue des Bouchers",Marseille,13008,France
5,17,Drachenblut Delikatessend,Sven Ottlieb,Walserweg 21,Aachen,52066,Germany
6,18,Du monde entier,Janine Labrune,"67, rue des Cinquante Otages",Nantes,44000,France
7,22,FISSA Fabrica Inter. Salchichas S.A.,Diego Roel,"C/ Moralzarzal, 86",Madrid,28034,Spain
8,23,Folies gourmandes,Martine Rancé,"184, chaussée de Tournai",Lille,59000,France
9,25,Frankenversand,Peter Franken,Berliner Platz 43,München,80805,Germany


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [419]:
tasks+=1
get_data(
    '''
SELECT Country, COUNT(*) as CNT FROM Customers
GROUP BY Country
Order by CNT DESC LIMit 3
    '''
)

,Country,CNT
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [420]:
tasks+=1
get_data('''
SELECT * FROM Orders
ORDER BY DATE (OrderDate)
LIMIT 1 OFFSET 9
''')

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10257,35,4,16.07.1996,3


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [421]:
tasks+=1
get_data('''
with max_sum as (
         select  sum(Price*Quantity) as sum_total,  OrderID  from Products p
         join OrderDetails o
         on p.productID = o.ProductID
         group by OrderID order by sum_total desc limit 1)
         select ord.OrderID , prod.ProductName, prod.price
         from  OrderDetails ord
         join products prod
         on prod.ProductID = ord.productid
         where ord.OrderID = (select orderid from max_sum)
        ''')

,OrderID,ProductName,Price
0,10372,Sir Rodney's Marmalade,81.0
1,10372,Côte de Blaye,263.5
2,10372,Camembert Pierrot,34.0
3,10372,Mozzarella di Giovanni,34.8


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [422]:
tasks+=1
get_data(''' with prod as (select ProductName,  sum(Quantity) as sum_q
         from orderdetails o join products p  on o.ProductID = p.ProductID
         group by ProductName order by sum_q desc limit 1)
        select ProductName, Quantity, orderid from orderdetails o join products p  on o.ProductID = p.ProductID
         where productname = (select productname from prod)
        ''')

,ProductName,Quantity,OrderID
0,Gorgonzola Telino,20,10253
1,Gorgonzola Telino,40,10272
2,Gorgonzola Telino,15,10273
3,Gorgonzola Telino,4,10325
4,Gorgonzola Telino,25,10335
5,Gorgonzola Telino,56,10342
6,Gorgonzola Telino,30,10356
7,Gorgonzola Telino,70,10359
8,Gorgonzola Telino,20,10363
9,Gorgonzola Telino,30,10374


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [423]:
tasks+=1
get_data(''' select SupplierName,Country,ContactName,Phone,count(*) as cnt
         from Products p 
         join suppliers s 
         on p.SupplierID = s.SupplierID
         group by SupplierName,Country,ContactName,Phone 
         order by cnt desc limit 5
         ''')

,SupplierName,Country,ContactName,Phone,cnt
0,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343,5
1,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755,5
2,New Orleans Cajun Delights,USA,Shelley Burke,(100) 555-4822,4
3,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448,4
4,Bigfoot Breweries,USA,Cheryl Saylor,(503) 555-9931,3


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [424]:
tasks+=1
get_data('''select CategoryName, sum(price*Quantity) as sum_ord ,Country
                from Categories c
                join products p on c.CategoryID = p.CategoryID
                join orderdetails o on o.productid = p.productid
                join orders on orders.orderid = o.orderid
                join Customers cust on cust.CustomerID = orders.CustomerID
                where Lower(cust.country) = 'brazil'
                group By CategoryName, Country
                order by sum_ord desc limit 1
         ''')

,CategoryName,sum_ord,Country
0,Beverages,13690.0,Brazil


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [425]:
tasks+=1
get_data(''' select max(sum_or) - min(sum_or) from (
         select sum(Price*Quantity) as sum_or, o.orderid,country
                from products p 
                join orderdetails o on o.productid = p.productid
                join orders on orders.orderid = o.orderid
                join Customers cust on cust.CustomerID = orders.CustomerID
                where Country ='USA'
                group By o.orderid,Country order by sum_or)
        ''')

,max(sum_or) - min(sum_or)
0,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [426]:
tasks+=1
get_data(''' select count(*) cnt,e.fio from orders o 
         join (select EmployeeID,  FirstName ||' '|| LastName fio  from employees e order by date(BirthDate) limit 3) e
         on o.EmployeeID = e.EmployeeID
         group by o.EmployeeID
        ''')

,cnt,fio
0,29,Nancy Davolio
1,20,Andrew Fuller
2,31,Janet Leverling


10. Сколько банок крабового мяса всего было заказано.

In [433]:
tasks+=1
get_data('''
        SELECT sum(Quantity)*24 as sum_tot, ProductName FROM Products join orderdetails 
        ON Products.ProductID = OrderDetails.ProductID
        WHERE ProductName LIKE '%Crab%'
                 ''')


,sum_tot,ProductName
0,6144,Boston Crab Meat


In [432]:
if tasks>=10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
